### Project Description

An international firm looking to expand its business across multiple countries has hired you as a Junior Data Engineer. Your primary task is to create an automated script that will extract, transform, and load (ETL) the GDP data of countries, as published by the International Monetary Fund (IMF). The data needs to be extracted from an online source and made accessible in two formats: a CSV file and a database, for easy access and analysis by the firm.

### Project Objective:
The goal of this project is to develop a Python script (`etl_project_gdp.py`) that performs the following tasks:

1. **Data Extraction**:
   - Retrieve the list of countries and their GDP in nominal terms (in Million USD) from a provided URL:
     - [List of countries by GDP (Nominal) on Wikipedia](https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29)

2. **Data Transformation**:
   - Convert the GDP values from "Million USD" to "Billion USD", rounding them to two decimal places.

3. **Data Loading**:
   - Save the transformed data in two formats:
     - A **CSV file**: `Countries_by_GDP.csv`
     - An **SQLite database**: `World_Economies.db`, with a table `Countries_by_GDP` containing the columns `Country` and `GDP_USD_billion`.

4. **Database Query**:
   - Run a query on the SQLite database to retrieve and display countries with a GDP greater than 100 billion USD.

5. **Logging**:
   - Log the entire process of the extraction, transformation, and loading steps in a log file: `etl_project_log.txt`, capturing key milestones and execution details with timestamps.

### Project Requirements:

- **Automation**: The script must be capable of running automatically to extract and update the GDP data whenever the IMF releases a new report (twice a year).
- **Data Storage**: Store the processed GDP data in both a CSV file and a relational database (SQLite).
- **Data Query**: Retrieve relevant data by running a query to display countries with GDPs exceeding 100 billion USD.
- **Logging**: Ensure the execution process is logged, including timestamps to track the progress of the ETL pipeline.

### Tools and Technologies:

- **Python** for scripting the ETL process.
- **Libraries**:
  - `requests` and `BeautifulSoup` for web scraping.
  - `pandas` for handling data and saving it to CSV.
  - `sqlite3` for interacting with the SQLite database.
  - `logging` for process logging with timestamps.
  
### Expected Outcomes:

1. **CSV File**: `Countries_by_GDP.csv` containing country names and GDP in billion USD.
2. **Database**: `World_Economies.db` with a table `Countries_by_GDP` storing the same data in a relational database format.
3. **Database Query**: A result set showing countries with GDP greater than 100 billion USD.
4. **Log File**: `etl_project_log.txt` capturing the full ETL process with detailed logs.

### Business Impact:
This automated process will help the firm to stay updated with the latest global economic rankings and support its decision-making when considering expansion opportunities in different countries. By automating the extraction and processing of the GDP data, the firm will save time and ensure accuracy as the IMF updates the data bi-annually.

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime

url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'

def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            if col[0].find('a') is not None and '—' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0],
                             "GDP_USD_millions": col[2].contents[0]}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df,df1], ignore_index=True)
    return df


def transform(df):
    GDP_list = df["GDP_USD_millions"].tolist()
    GDP_list = [float("".join(x.split(','))) for x in GDP_list]
    GDP_list = [np.round(x/1000,2) for x in GDP_list]
    df["GDP_USD_millions"] = GDP_list
    df=df.rename(columns = {"GDP_USD_millions":"GDP_USD_billions"})
    return df


def load_to_csv(df, csv_path):
    df.to_csv(csv_path)


def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)


def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)


def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("./etl_project_log.txt","a") as f:
        f.write(timestamp + ' : ' + message + '\n')


log_progress('Preliminaries complete. Initiating ETL process')

df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')

df = transform(df)

log_progress('Data transformation complete. Initiating loading process')

load_to_csv(df, csv_path)

log_progress('Data saved to CSV file')

sql_connection = sqlite3.connect('World_Economies.db')

log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)

log_progress('Data loaded to Database as table. Running the query')

query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(query_statement, sql_connection)

log_progress('Process Complete.')

sql_connection.close()

SELECT * from Countries_by_GDP WHERE GDP_USD_billions >= 100
          Country  GDP_USD_billions
0   United States          26854.60
1           China          19373.59
2           Japan           4409.74
3         Germany           4308.85
4           India           3736.88
..            ...               ...
64          Kenya            118.13
65         Angola            117.88
66           Oman            104.90
67      Guatemala            102.31
68       Bulgaria            100.64

[69 rows x 2 columns]
